# 05 - Model Comparison Dashboard

This notebook provides an easy comparison of all models trained in notebook 3:
- Loads model results from notebook 3
- Side-by-side performance comparison
- Visual comparison of model outputs
- Clinical decision support metrics


In [ ]:
# Ensure repository root on sys.path for `import app.*`
import sys
from pathlib import Path
repo_root = (Path.cwd() / '..').resolve()
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))
print('Repo root:', repo_root)


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import CoxPHFitter
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

from app.evaluation import compute_concordance_index, compute_td_auc
from app.feature_engineering import engineer_features

print("Libraries loaded successfully!")


In [ ]:
# Load model results from notebook 3
import pickle
with open('../models/model_results.pkl', 'rb') as f:
    model_results = pickle.load(f)

# Extract data
models = model_results['models']
c_indices = model_results['c_indices']
comparison_df = model_results['comparison_df']
X_test = model_results['X_test']
y_test = model_results['y_test']

print("Model results loaded successfully!")
print(f"Available models: {list(models.keys())}")
print(f"Test set shape: {X_test.shape}")
print(f"Event rate: {y_test['event'].mean():.3f}")
print("\nModel Performance Summary:")
print(comparison_df)


## Train Multiple Models


In [ ]:
# 1. Cox Proportional Hazards
print("Training Cox Proportional Hazards...")
cox_train = X_train.copy()
cox_train['time_to_event'] = y_train['time_to_event'].values
cox_train['event'] = y_train['event'].values

cox_test = X_test.copy()
cox_test['time_to_event'] = y_test['time_to_event'].values
cox_test['event'] = y_test['event'].values

cph = CoxPHFitter(penalizer=0.1, l1_ratio=0.1)
cph.fit(cox_train, duration_col='time_to_event', event_col='event', show_progress=False)
cox_risk_scores = cph.predict_partial_hazard(cox_test)
print("✓ Cox PH trained")


In [ ]:
# 2. XGBoost Survival (Cox objective)
print("Training XGBoost Survival (Cox)...")
dtrain_cox = xgb.DMatrix(X_train, label=y_train['time_to_event'].values)
dtest_cox = xgb.DMatrix(X_test, label=y_test['time_to_event'].values)

params_cox = {
    'objective': 'survival:cox',
    'eval_metric': 'cox-nloglik',
    'eta': 0.05,
    'max_depth': 3,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': 42,
}

xgb_cox = xgb.train(params_cox, dtrain_cox, num_boost_round=300, verbose_eval=False)
xgb_cox_risk_scores = xgb_cox.predict(dtest_cox)
print("✓ XGBoost Cox trained")


In [ ]:
# 3. XGBoost Survival (AFT objective)
print("Training XGBoost Survival (AFT)...")
dtrain_aft = xgb.DMatrix(X_train, label=y_train['time_to_event'].values)
dtest_aft = xgb.DMatrix(X_test, label=y_test['time_to_event'].values)

params_aft = {
    'objective': 'survival:aft',
    'eval_metric': 'aft-nloglik',
    'eta': 0.05,
    'max_depth': 3,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'seed': 42,
}

xgb_aft = xgb.train(params_aft, dtrain_aft, num_boost_round=300, verbose_eval=False)
xgb_aft_risk_scores = xgb_aft.predict(dtest_aft)
print("✓ XGBoost AFT trained")


In [ ]:
# 4. Random Forest (classification proxy)
print("Training Random Forest...")
rf = RandomForestClassifier(n_estimators=300, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train['event'])
rf_proba = rf.predict_proba(X_test)[:, 1]
print("✓ Random Forest trained")


## Model Performance Comparison


In [ ]:
# Compute C-index for all models
models = {
    'Cox PH': cox_risk_scores,
    'XGBoost Cox': xgb_cox_risk_scores,
    'XGBoost AFT': xgb_aft_risk_scores,
    'Random Forest': rf_proba
}

c_indices = {}
for name, scores in models.items():
    c_index = compute_concordance_index(y_test['event'], y_test['time_to_event'], scores)
    c_indices[name] = c_index
    print(f"{name} C-index: {c_index:.3f}")

# Create comparison table
comparison_df = pd.DataFrame([
    {'Model': name, 'C-index': c_index}
    for name, c_index in c_indices.items()
]).sort_values('C-index', ascending=False)

print("\nModel Performance Ranking:")
print(comparison_df)


In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# C-index comparison
axes[0, 0].bar(comparison_df['Model'], comparison_df['C-index'], color='steelblue', alpha=0.7)
axes[0, 0].set_title('C-index Comparison')
axes[0, 0].set_ylabel('C-index')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(True, alpha=0.3)

# Risk score distributions
axes[0, 1].hist(cox_risk_scores, bins=20, alpha=0.7, label='Cox PH', density=True)
axes[0, 1].hist(xgb_cox_risk_scores, bins=20, alpha=0.7, label='XGBoost Cox', density=True)
axes[0, 1].hist(xgb_aft_risk_scores, bins=20, alpha=0.7, label='XGBoost AFT', density=True)
axes[0, 1].hist(rf_proba, bins=20, alpha=0.7, label='Random Forest', density=True)
axes[0, 1].set_title('Risk Score Distributions')
axes[0, 1].set_xlabel('Risk Score')
axes[0, 1].set_ylabel('Density')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Time-dependent AUC comparison
time_points = np.linspace(1, 30, 10)
td_auc_results = {}

for name, scores in models.items():
    td_auc_scores = []
    for t in time_points:
        auc_t = compute_td_auc(y_test['event'], y_test['time_to_event'], scores, t)
        td_auc_scores.append(auc_t)
    td_auc_results[name] = td_auc_scores

for name, scores in td_auc_results.items():
    axes[1, 0].plot(time_points, scores, 'o-', label=name, linewidth=2, markersize=4)

axes[1, 0].axhline(y=0.5, color='r', linestyle='--', alpha=0.7, label='Random')
axes[1, 0].set_title('Time-Dependent AUC')
axes[1, 0].set_xlabel('Time (days)')
axes[1, 0].set_ylabel('AUC')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Model correlation matrix
risk_scores_df = pd.DataFrame({
    'Cox PH': cox_risk_scores,
    'XGBoost Cox': xgb_cox_risk_scores,
    'XGBoost AFT': xgb_aft_risk_scores,
    'Random Forest': rf_proba
})

correlation_matrix = risk_scores_df.corr()
im = axes[1, 1].imshow(correlation_matrix, cmap='coolwarm', vmin=-1, vmax=1)
axes[1, 1].set_title('Model Correlation Matrix')
axes[1, 1].set_xticks(range(len(correlation_matrix.columns)))
axes[1, 1].set_yticks(range(len(correlation_matrix.columns)))
axes[1, 1].set_xticklabels(correlation_matrix.columns, rotation=45)
axes[1, 1].set_yticklabels(correlation_matrix.columns)

# Add correlation values to the plot
for i in range(len(correlation_matrix.columns)):
    for j in range(len(correlation_matrix.columns)):
        text = axes[1, 1].text(j, i, f'{correlation_matrix.iloc[i, j]:.2f}',
                              ha="center", va="center", color="black", fontweight='bold')

plt.colorbar(im, ax=axes[1, 1])
plt.tight_layout()
plt.show()
